In [16]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import pandas_helper
from pandas_helper import Concepts
pd.set_option('display.max_rows', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = pandas_helper.load_data("data/wikilanguage.tsv")

In [41]:
films = df[pandas_helper.instance_of(df, Concepts.FILM)]
pandas_helper.top_by_year(films, top_col='frwiki_pagerank')

,sample_label,coord_latitude,coord_longitude,coord_altitude,coord_precision,country_of_origin,publication_date,frwiki_title,frwiki_pagerank,arwiki_title,...,jawiki_title,jawiki_pagerank,ruwiki_title,ruwiki_pagerank,eswiki_title,eswiki_pagerank,zhwiki_title,zhwiki_pagerank,direct_instance_of,instance_of
concept_id,,,,,,,,,,,,,,,,,,,,,
Q3604746,Avatar 2,NaN,NaN,NaN,NaN,Q30,2021-12-17,Avatar 2,2.579254e-07,أفاتار 2,...,NaN,NaN,Аватар 2,2.910704e-07,Avatar 2,2.477207e-07,阿凡達2,2.190046e-07,Q18011172,"Q223557,Q1150070,Q22294683,Q11424,Q18011172,Q1..."
Q57177410,Birds of Prey,NaN,NaN,NaN,NaN,Q30,2020-01-01,"Birds of Prey (film, 2020)",4.212913e-07,NaN,...,ハーレイ・クインの華麗なる覚醒 Birds of Prey,1.956056e-07,Хищные птицы (фильм),1.430527e-07,Birds of Prey (película),2.301406e-07,猛禽小隊：小丑女大解放,2.194810e-07,"Q11424,Q18011172","Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q23781155,Avengers: Endgame,NaN,NaN,NaN,NaN,Q30,2019-04-22,Avengers: Endgame,1.080542e-06,المنتقمون: نهاية اللعبة,...,アベンジャーズ/エンドゲーム,1.148479e-06,Мстители: Финал,1.167474e-06,Avengers: Endgame,1.814679e-06,復仇者聯盟：終局之戰,1.305208e-06,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q23780914,Avengers: Infinity War,NaN,NaN,NaN,NaN,Q30,2018-04-25,Avengers: Infinity War,9.816594e-07,المنتقمون: الحرب اللانهائية,...,アベンジャーズ/インフィニティ・ウォー,1.118511e-06,Мстители: Война бесконечности,1.096987e-06,Avengers: Infinity War,1.684145e-06,復仇者聯盟3：無限之戰,1.616315e-06,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q18486021,Star Wars: Episode VIII – The Last Jedi,NaN,NaN,NaN,NaN,Q30,2017-12-09,"Star Wars, épisode VIII : Les Derniers Jedi",1.040272e-06,حرب النجوم: الجيداي الأخير,...,スター・ウォーズ/最後のジェダイ,1.309468e-06,Звёздные войны: Последние джедаи,8.215371e-07,Star Wars: Episodio VIII - Los últimos Jedi,9.293657e-07,STAR WARS：最後的絕地武士,9.296590e-07,"Q11424,Q229390","Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q23013699,Lion,NaN,NaN,NaN,NaN,Q30,2016-09-10,"Lion (film, 2016)",2.533000e-06,ليون (فيلم 2016),...,LION/ライオン 〜25年目のただいま〜,3.520093e-07,"Лев (фильм, 2016)",2.456915e-07,Lion (película),3.229595e-07,漫漫回家路 (2016年電影),3.882009e-07,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q6074,Star Wars Episode VII: The Force Awakens,NaN,NaN,NaN,NaN,Q30,2015-12-16,"Star Wars, épisode VII : Le Réveil de la Force",2.252213e-06,حرب النجوم: القوة تنهض,...,スター・ウォーズ/フォースの覚醒,2.283360e-06,Звёздные войны: Пробуждение силы,1.630040e-06,Star Wars: Episodio VII - El despertar de la F...,2.288886e-06,STAR WARS：原力覺醒,1.716167e-06,"Q229390,Q24869","Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q1765358,Captain America: The Winter Soldier,NaN,NaN,NaN,NaN,Q30,2014-03-26,Captain America : Le Soldat de l'hiver,1.488842e-06,كابتن أمريكا: جندي الشتاء,...,キャプテン・アメリカ/ウィンター・ソルジャー,1.515979e-06,Первый мститель: Другая война,8.086946e-07,Captain America: The Winter Soldier,7.963721e-07,美國隊長2：酷寒戰士,1.094598e-06,"Q11424,Q229390","Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q3023357,12 Years a Slave,NaN,NaN,NaN,NaN,Q145,2013-08-30,Twelve Years a Slave,1.358855e-06,عبد لاثني عشر عاما (فيلم),...,それでも夜は明ける,1.684607e-06,12 лет рабства,1.714417e-06,12 años de esclavitud,1.004526e-06,为奴十二年,1.529730e-06,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."


In [75]:
japan_films = films[pandas_helper.country_of_origin(films, pandas_helper.resolve_label(df, "France"))]
japan_films.nlargest(20, 'enwiki_pagerank')[['sample_label']]

,sample_label
concept_id,
Q334780,Moonraker
Q18407,La Dolce Vita
Q244971,A Trip to the Moon
Q171861,The Artist
Q146673,Basic Instinct
Q501874,"O Brother, Where Art Thou?"
Q150804,The Pianist
Q41132,Barbarella
Q106506,The Fifth Element


In [59]:
japan_prefers = pandas_helper.kl_divergence(films, base_wiki="enwiki", target_wiki="jawiki")
pandas_helper.top_by_year(japan_prefers, top_col='kl_divergence')[
    ['publication_date', 'enwiki_title', 'jawiki_title', 'kl_divergence', 'kl_relative_to_max']
]

,publication_date,enwiki_title,jawiki_title,kl_divergence,kl_relative_to_max
concept_id,,,,,
Q6434144,2020-01-01,Evangelion: 3.0+1.0,シン・エヴァンゲリオン劇場版,4.712089e-05,0.016783
Q59692464,2019-07-19,Weathering with You,天気の子,3.334622e-04,0.118766
Q45865176,2018-07-20,Mirai (film),未来のミライ,2.662880e-04,0.094841
Q24868434,2017-03-18,Kuroko's Basketball The Movie: Last Game,黒子のバスケ (アニメ),6.665888e-04,0.237412
Q21072514,2016-07-16,High & Low: The Movie,HiGH&LOW〜THE STORY OF S.W.O.R.D.〜,1.604605e-03,0.571495
Q18729861,2015-02-28,Maku ga Agaru,幕が上がる,1.531050e-03,0.545298
Q30240458,2014-01-01,Kokuhaku Jikkō Iinkai: Ren'ai Series,告白実行委員会〜恋愛シリーズ〜,1.009588e-03,0.359575
Q11550375,2013-01-01,The Eternal Zero,永遠の0,5.043193e-04,0.179618
Q474033,2012-04-03,Battleship (film),バトルシップ (映画),8.197806e-04,0.291973


In [51]:
japan_prefers

,sample_label,coord_latitude,coord_longitude,coord_altitude,coord_precision,country_of_origin,publication_date,frwiki_title,frwiki_pagerank,arwiki_title,...,ruwiki_pagerank,eswiki_title,eswiki_pagerank,zhwiki_title,zhwiki_pagerank,direct_instance_of,instance_of,kl_divergence,odds_ratio,kl_relative_to_max
concept_id,,,,,,,,,,,,,,,,,,,,,
Q1060124,Castle of Sand,NaN,NaN,NaN,NaN,Q17,1961-01-01,NaN,NaN,NaN,...,1.148906e-07,NaN,NaN,砂之器,7.078664e-07,"Q11424,Q7725634","Q223557,Q7725634,Q47461344,Q1150070,Q22294683,...",0.002808,24.418205,1.000000
Q698745,Mobile Suit Zeta Gundam,NaN,NaN,NaN,NaN,Q17,2004-01-01,Mobile Suit Zeta Gundam,4.066470e-07,NaN,...,2.924490e-07,Mobile Suit Zeta Gundam,5.384968e-07,機動戰士Z GUNDAM,2.275101e-06,"Q11424,Q5398426,Q21191019","Q3695082,Q332624,Q1151067,Q694975,Q2431196,Q10...",0.002405,7.365762,0.856505
Q11659568,Minami no Teiō,NaN,NaN,NaN,NaN,Q17,1990-01-01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Q11424,Q21198342","Q223557,Q1004,Q47461344,Q1150070,Q22294683,Q11...",0.001844,33.877477,0.656675
Q3235913,Three Outlaw Samurai,NaN,NaN,NaN,NaN,Q17,1964-01-01,Les Trois Samouraïs hors-la-loi,1.196903e-07,NaN,...,7.041893e-08,NaN,NaN,NaN,NaN,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15...",0.001812,29.184836,0.645443
Q21072514,HiGH&LOW,NaN,NaN,NaN,NaN,Q17,2016-07-16,NaN,NaN,NaN,...,NaN,NaN,NaN,HiGH&LOW 熱血街頭,3.750587e-07,"Q11424,Q5398426","Q3695082,Q332624,Q1151067,Q694975,Q2431196,Q10...",0.001605,24.263306,0.571495
Q18729861,Maku ga Agaru,NaN,NaN,NaN,NaN,Q17,2015-02-28,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15...",0.001531,25.246971,0.545298
Q7250018,Proof of the Man,NaN,NaN,NaN,NaN,Q17,1977-01-01,NaN,NaN,NaN,...,1.026650e-07,NaN,NaN,人性的证明,3.896287e-07,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15...",0.001146,15.246742,0.408037
Q21697406,Your Name.,NaN,NaN,NaN,NaN,Q17,2016-07-03,Your Name.,2.454315e-07,اسمك (فيلم),...,2.378122e-07,Kimi no Na wa.,2.888284e-07,你的名字。,1.678653e-06,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15...",0.001126,6.024994,0.400861
Q202681,Joint Security Area,NaN,NaN,NaN,NaN,Q884,2000-09-09,Joint Security Area (film),2.252005e-07,المنطقة الأمنية المشتركة,...,3.008901e-07,Área común de seguridad,1.674424e-07,JSA安全地帶,6.879013e-07,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15...",0.001124,8.229986,0.400330


In [46]:
base_wiki = "enwiki"
target_wiki = "jawiki"
matching_rows = films.loc[
    films[f"{base_wiki}_title"].notna() & films[f"{target_wiki}_title"].notna()
].copy()
matching_rows

,sample_label,coord_latitude,coord_longitude,coord_altitude,coord_precision,country_of_origin,publication_date,frwiki_title,frwiki_pagerank,arwiki_title,...,jawiki_title,jawiki_pagerank,ruwiki_title,ruwiki_pagerank,eswiki_title,eswiki_pagerank,zhwiki_title,zhwiki_pagerank,direct_instance_of,instance_of
concept_id,,,,,,,,,,,,,,,,,,,,,
Q595,The Intouchables,NaN,NaN,NaN,NaN,Q142,2011-09-23,Intouchables (film),2.429299e-06,المنبوذون (فيلم),...,最強のふたり,1.078146e-06,1+1 (фильм),6.374907e-07,Intouchables,3.191606e-07,无法触碰 (法国电影),1.300725e-06,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q2345,12 Angry Men,NaN,NaN,NaN,NaN,Q30,1957-04-10,Douze hommes en colère (film),5.490036e-07,12 رجلا غاضبا,...,十二人の怒れる男,2.002176e-06,"12 разгневанных мужчин (фильм, 1957)",1.198549e-06,12 Angry Men (película de 1957),8.707204e-07,十二怒漢 (電影),6.448246e-07,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q4466,Titanic II,NaN,NaN,NaN,NaN,Q30,2010-01-01,Titanic : Odyssée 2012,9.844248e-08,NaN,...,タイタニック2012,2.127719e-07,Титаник 2 (фильм),1.525852e-07,Titanic II,2.580889e-07,NaN,NaN,Q506240,"Q223557,Q1150070,Q732577,Q11424,Q15621286,Q191..."
Q7216,Pirates II: Stagnetti's Revenge,NaN,NaN,NaN,NaN,Q30,2008-09-27,Pirates 2 : La Vengeance de Stagnetti,1.335662e-07,NaN,...,パイレーツ・オブ・アイランド,1.783875e-07,Пираты II: Месть Стагнетти,1.490775e-07,Pirates II: Stagnetti's Revenge,1.251286e-07,加勒比女海盗2：斯塔涅蒂的报复,1.085587e-07,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q11618,Eraserhead,NaN,NaN,NaN,NaN,Q30,1977-01-01,Eraserhead,2.742158e-07,رأس ممحاة (فيلم),...,イレイザーヘッド,3.345753e-07,Голова-ластик,5.176870e-07,Eraserhead,3.385932e-07,橡皮头,2.948331e-07,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q60987790,Onna Gokuakuchō,NaN,NaN,NaN,NaN,Q17,1970-01-01,NaN,NaN,NaN,...,おんな極悪帖,1.119159e-07,NaN,NaN,NaN,NaN,NaN,NaN,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q61948516,King of Prism: Shiny Seven Stars,NaN,NaN,NaN,NaN,Q17,2019-03-02,NaN,NaN,NaN,...,KING OF PRISM -Shiny Seven Stars-,4.793796e-07,NaN,NaN,NaN,NaN,NaN,NaN,"Q11424,Q1107","Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."
Q62082323,The Kill Team,NaN,NaN,NaN,NaN,Q30,2019-01-01,NaN,NaN,NaN,...,ザ・キル・チーム,9.777686e-08,NaN,NaN,NaN,NaN,NaN,NaN,Q11424,"Q223557,Q1150070,Q22294683,Q11424,Q1914636,Q15..."


In [14]:
pandas_helper.resolve_label(df, "Canaaada")

AssertionError: Too many or few rows found (0)

In [ ]:
dataset[dataset.loc[dataset['sample_label'] == 'George Washington']]

In [ ]:
# Memory Hungry Function
top_concepts = pandas_helper.best_concepts(
    dataset, sample=0.1, n=500
)

In [ ]:
films = instance_of(dataset, Concepts.FILM).drop(
    index=[
        "Q2537294", # All-Russia State Television and Radio Broadcas
        "Q11659568", # Minami no Teiō 
    ]
)

In [ ]:
games = instance_of(dataset, Concepts.VIDEO_GAME).drop(
    index=[
        
    ]
)

In [ ]:
top_ranked(games, "frwiki")

In [ ]:
marginals = top_kl(films, "enwiki", "jawiki", marginals=True, importance_weight=1)["enwiki_title"].reset_index()
no_marginals = top_kl(films, "enwiki", "jawiki", marginals=True, importance_weight=2)["enwiki_title"].reset_index()
pandas.concat((marginals, no_marginals), axis=1)